<a href="https://colab.research.google.com/github/ayanand/CapstoneML/blob/main/analysiscap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/ayanand/CapstoneML/blob/main/analysiscap.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

In [35]:
import pandas as pd
!pip install fsspec
!pip install gcsfs 
 
 


In [44]:
df = pd.read_csv('gs://capstoragebucket2020/All_joined_v4.csv')

In [43]:
df.tail()

,latitude,longitude,quarter,year,incendie_count,alarmes_incendies_count,total_crimes,vols_count,mefait_count,vol_de_vehicule_count,introduction_count,vol_moteur_count,infractions_entrainant_count,sum_etage_hors_sol,sum_nombre_logement,min_annee_construction,max_annee_construction,avg_annee_construction,sum_superficie_terrain,area,population,dwellings,households,averagehouseholdsize,averageage,averagesizeofcensusfamilies,workers,caserne_count,fire,distance
11709,45.67,-73.50,4,1,0,0,2,0,0,0,1,1,0,1706,2237,1914,2019,1969.399127,518847,0.11503,505,285,273,1.8,43.2,2.6,100,0,0,0
11738,45.68,-73.50,4,1,0,0,5,0,1,3,0,1,0,1377,898,1959,2019,1994.756098,465540,0.45534,821,422,386,2.1,37.5,2.7,335,0,0,0
11760,45.69,-73.50,4,1,0,0,2,0,0,0,2,0,0,254,285,1959,2018,1999.281967,75473,2.99229,1350,519,518,2.6,38.5,2.9,580,0,0,0
11784,45.69,-73.49,4,1,0,0,3,0,0,1,0,2,0,1754,1687,1925,2017,1990.051636,493065,0.20290,677,290,286,2.4,40.4,2.7,195,0,0,0
11819,45.70,-73.49,4,1,0,0,1,0,0,0,0,1,0,933,805,1900,2019,1997.866756,177975,0.48567,645,255,249,2.6,41.3,2.9,170,1,0,0


In [45]:
df=df.sort_values(by=['year', 'quarter','latitude','longitude'])

In [46]:
df['incendie_count'].value_counts()


#Define three classes of fire risk by the summed number of fire incidence from 2015-2020
#low risk < 5, moderate risk 5-15, high risk > 15
df['fire'] = df['incendie_count']

df['fire'][df['incendie_count']==0] = 0
df['fire'][(df['incendie_count']>=1) & (df['incendie_count']<4)] = 1
df['fire'][df['incendie_count']>=4] = 2



  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [47]:
df_new = df
df_new['year'] =2021- df_new['year']

In [48]:
import geopy.distance

def dist(x,y):
  coords_1 = (45.504654, -73.56546)
  coords_2 = (x, y)
  return geopy.distance.vincenty(coords_1, coords_2).km


In [49]:
df_new['distance']=0

In [50]:
for x in range(0,len(df_new)-1):
  df_new['distance'][x]=dist(df_new['latitude'][x], df_new['longitude'][x])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [51]:
df_new.tail(10)

,latitude,longitude,quarter,year,incendie_count,alarmes_incendies_count,total_crimes,vols_count,mefait_count,vol_de_vehicule_count,introduction_count,vol_moteur_count,infractions_entrainant_count,sum_etage_hors_sol,sum_nombre_logement,min_annee_construction,max_annee_construction,avg_annee_construction,sum_superficie_terrain,area,population,dwellings,households,averagehouseholdsize,averageage,averagesizeofcensusfamilies,workers,caserne_count,fire,distance
11545,45.66,-73.51,4,1,0,0,5,0,0,4,0,1,0,1207,1044,1941,2017,1984.169195,494180,0.28223,448,168,167,2.6,41.7,3.0,135,0,0,17
11569,45.66,-73.50,4,1,0,0,2,0,1,1,0,0,0,1953,1624,1925,2014,1976.802276,557374,0.30364,1090,395,388,2.8,39.9,3.0,290,0,0,18
11593,45.66,-73.49,4,1,0,0,1,0,0,1,0,0,0,490,542,1752,2018,1976.853933,178758,0.21275,487,212,210,2.2,47.9,2.6,110,0,0,18
11617,45.67,-73.55,4,1,0,0,1,0,0,1,0,0,0,874,607,1900,2017,1994.664671,196023,0.72304,453,158,154,2.7,43.1,2.7,75,0,0,18
11685,45.67,-73.51,4,1,0,2,2,0,0,0,0,2,0,459,576,1920,2015,1981.962264,338851,0.73267,720,345,343,2.0,47.3,2.7,165,1,0,18
11709,45.67,-73.50,4,1,0,0,2,0,0,0,1,1,0,1706,2237,1914,2019,1969.399127,518847,0.11503,505,285,273,1.8,43.2,2.6,100,0,0,19
11738,45.68,-73.50,4,1,0,0,5,0,1,3,0,1,0,1377,898,1959,2019,1994.756098,465540,0.45534,821,422,386,2.1,37.5,2.7,335,0,0,20
11760,45.69,-73.50,4,1,0,0,2,0,0,0,2,0,0,254,285,1959,2018,1999.281967,75473,2.99229,1350,519,518,2.6,38.5,2.9,580,0,0,21
11784,45.69,-73.49,4,1,0,0,3,0,0,1,0,2,0,1754,1687,1925,2017,1990.051636,493065,0.20290,677,290,286,2.4,40.4,2.7,195,0,0,21
11819,45.70,-73.49,4,1,0,0,1,0,0,0,0,1,0,933,805,1900,2019,1997.866756,177975,0.48567,645,255,249,2.6,41.3,2.9,170,1,0,22


In [52]:

df_new = df_new.drop("latitude", axis=1)
df_new = df_new.drop("longitude", axis=1)

In [53]:
df_new = df_new.drop("min_annee_construction", axis=1)
df_new = df_new.drop("max_annee_construction", axis=1)
df_new = df_new.drop("incendie_count", axis=1)

In [54]:
#y= df_new["incendie_count"].copy()
y= df_new["fire"].copy()
X=df_new.drop("fire", axis=1)

In [55]:
y.value_counts()

0    7633
1    3967
2     243
Name: fire, dtype: int64

In [56]:
len(X),len(y)

(11843, 11843)

In [57]:
corr_matrix = df_new.corr()
corr_matrix["fire"].sort_values(ascending=False)

fire                            1.000000
sum_nombre_logement             0.479032
total_crimes                    0.429216
sum_etage_hors_sol              0.420503
alarmes_incendies_count         0.410367
mefait_count                    0.399054
introduction_count              0.395315
vols_count                      0.312964
vol_moteur_count                0.309208
vol_de_vehicule_count           0.209143
caserne_count                   0.158662
sum_superficie_terrain          0.084048
dwellings                       0.061831
infractions_entrainant_count    0.052661
households                      0.036779
year                            0.014774
quarter                        -0.031297
averageage                     -0.038386
population                     -0.051500
averagesizeofcensusfamilies    -0.071044
workers                        -0.083336
averagehouseholdsize           -0.167736
area                           -0.202429
avg_annee_construction         -0.207849
distance        

In [58]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

scaler = StandardScaler()
scaler.fit(X)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [59]:
X=scaler.transform(X)

In [70]:
X

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,-1.337680,1.455784,-0.731833,-0.655565,-0.432903,-0.675203,-0.321695,-0.465661,-0.590038,-0.097931,-0.620745,-0.626044,-1.180331,-0.977545,-0.438656,-0.829859,-1.246526,-1.376535,-3.705581,5.128243,-4.794451,-1.199589,-0.381403,2.714514
1,-1.337680,1.455784,-0.615674,-0.785484,-0.432903,-0.675203,-0.772084,-0.631346,-0.590038,-0.097931,-1.260236,-1.027951,-3.159169,-2.271987,2.334588,0.817266,0.105745,0.165279,0.894062,-0.318333,0.436879,1.221259,-0.381403,2.714514
2,-1.337680,1.455784,-0.731833,-0.525645,-0.432903,-0.675203,-0.321695,-0.134291,-0.590038,-0.097931,-0.924547,-0.881451,-0.687748,0.029095,0.432053,-0.188305,-0.284027,-0.267349,0.418237,0.433956,0.268126,-0.204720,-0.381403,2.571448
3,-1.337680,1.455784,-0.731833,-0.720524,-0.432903,-0.675203,-0.772084,-0.465661,-0.590038,-0.097931,-0.954662,-0.904670,-0.343020,-0.146771,-0.357123,-0.310985,-0.558459,-0.580315,0.576845,1.065879,0.099374,-0.370531,-0.381403,2.428383
4,-1.337680,1.455784,-0.615674,-0.785484,-0.432903,-0.675203,-0.772084,-0.631346,-0.590038,-0.097931,-1.162806,-1.001415,1.894999,0.014570,0.432053,-0.188305,-0.284027,-0.267349,0.418237,0.433956,0.268126,-0.204720,-0.381403,2.571448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11838,1.359927,-1.491975,-0.731833,-0.655565,-0.432903,-0.675203,-0.772084,-0.465661,-0.403681,-0.097931,0.245491,0.207069,-0.065885,0.561445,-0.467969,-0.445731,-0.140845,-0.120072,-0.850630,0.273468,-0.406884,-0.536343,-0.381403,0.997727
11839,1.359927,-1.491975,-0.731833,-0.460685,-0.432903,-0.426990,0.579082,-0.631346,-0.403681,-0.097931,-0.045911,-0.533169,1.389358,0.252226,-0.363949,0.189790,0.404041,0.400003,-0.374805,-0.298272,-0.238132,1.022285,-0.381403,1.140792
11840,1.359927,-1.491975,-0.731833,-0.655565,-0.432903,-0.675203,-0.772084,-0.299976,-0.590038,-0.097931,-1.040577,-0.872053,1.649099,-2.010444,0.411506,1.253684,0.789836,1.007524,0.418237,-0.197967,0.099374,2.647238,-0.381403,1.283858
11841,1.359927,-1.491975,-0.731833,-0.590605,-0.432903,-0.675203,-0.321695,-0.631346,-0.217325,-0.097931,0.288006,-0.096987,1.119368,0.411890,-0.441111,-0.099815,-0.120959,-0.060240,0.101020,-0.007387,-0.238132,0.093741,-0.381403,1.283858


In [71]:
X=pd.DataFrame(X)

In [62]:
X_train=X[df_new['year'] > 2] # choose 2019 data as train
X_test =X[df_new['year'] == 2] # choose 2019 data as validation
y_train = y[df_new['year'] > 2]
y_test = y[df_new['year'] == 2]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [76]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=4)

len(X_train),len(X_test),len(y_train),len(y_test)

(9474, 2369, 9474, 2369)

In [77]:
y_train.value_counts(), y_test.value_counts()

(0    6088
 1    3190
 2     196
 Name: fire, dtype: int64, 0    1545
 1     777
 2      47
 Name: fire, dtype: int64)

In [78]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [79]:
lin_reg.coef_

array([-0.01897504,  0.00143743,  0.06965785,  0.01374782,  0.00798473,
        0.04180543,  0.01188125,  0.01429123, -0.01516217, -0.00416957,
       -0.02133139,  0.13960683, -0.00609208,  0.00342418, -0.02444003,
        0.0370121 , -0.03242547,  0.01248774, -0.04995773, -0.03232264,
        0.05802084, -0.03715717,  0.0066432 , -0.02205765])

In [80]:
print(lin_reg.score(X_train, y_train))

0.259164032110604


In [81]:

print(lin_reg.score(X_test,y_test))

0.25054246528146895


In [82]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42,max_depth=5)
tree_reg.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=5,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='best')

In [83]:
from sklearn.metrics import mean_squared_error
import numpy as np

y_predictions = tree_reg.predict(X_test)
tree_mse = mean_squared_error(y_test, y_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.4535348481859323

In [84]:
Xyz=pd.DataFrame(y_predictions)

Xyz.value_counts(), y_test.value_counts()

(0.128059    451
 0.353621    330
 0.557538    292
 0.206512    268
 0.928025    160
 0.080670    159
 0.670487    158
 0.196147    141
 0.387560     63
 0.186335     49
 0.867647     33
 0.250000     33
 0.500000     32
 0.011905     31
 0.210526     31
 1.096774     26
 0.479167     22
 1.393617     21
 0.317073     19
 0.616667     12
 0.309859     10
 0.950000      7
 0.111111      7
 0.937500      5
 1.000000      4
 0.454545      3
 0.666667      2
 dtype: int64, 0    1545
 1     777
 2      47
 Name: fire, dtype: int64)

In [85]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=10, random_state=42, max_depth=10)
forest_reg.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [86]:
print(str(forest_reg.score(X_train, y_train)) + " "+ str(forest_reg.score(X_test, y_test) ))

0.5167125782691818 0.23486385039973592


In [88]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
# prepare the cross-validation procedure
cv = KFold(n_splits=5, random_state=1, shuffle=True)
# create model
xgboost = XGBClassifier()
# evaluate model
xgboost_scores = cross_val_score(xgboost, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (mean(xgboost_scores ), std(xgboost_scores )))

Accuracy: 0.720 (0.008)


In [89]:

xgboost.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [90]:
y_test_pred = xgboost.predict(X_test)

In [91]:

print('Accuracy of XGboost classifier on test set: {:.2f}'.format(xgboost.score(X_test, y_test)))

Accuracy of XGboost classifier on test set: 0.72


In [92]:
from sklearn.metrics import confusion_matrix
confusion_matrix_valid = confusion_matrix(y_test, y_test_pred)
print(confusion_matrix_valid)

[[1347  198    0]
 [ 408  367    2]
 [   2   45    0]]
